# import data & Pre-processing

In [1]:
import pandas as pd
df = pd.read_csv("C:/Users/user/RPA/book_recording_sample.csv") #일단 불러옴
#전처리
#pd.options.display.max_colwidth = 50
#필요 없는 줄 날리기, 세번째 행부터, 6번째 열까지만 가져오기
df = pd.DataFrame(df.values[3:,:6])
df.columns = ["name","section", "year", "grade","sem","book"]#column 이름 지정
df = df.dropna(how='all')#모든 칸이 nan인 행 지우기
df = df.fillna(method='ffill') #행별로 이름 채우기(전 행의 이름과 동일함)
df.drop(df[df['name'] =='이  름'].index, inplace = True) #페이지 넘어갈 때 있는 열이름 삭제
original = df.values.tolist() #list로
pd.options.display.max_colwidth = 100
df.head(3)

,name,section,year,grade,sem,book
0,강철수,공통,2018,1,1,"서민의 기생충 열전(서미), 과학 일시정지(가치를 꿈꾸는 과학교사모임), 모든 순간의 물리학(카를로 로벨리), 미래학자의 인공지능 시나리오(최윤식), 내 휴대폰 속의 슈퍼스..."
1,강철수,수학,2018,1,2,수학의 역사(지즈강)
2,강철수,공통,2018,1,2,"완득이(김려령), 나미야 잡화점의 기적(히가시노 게이고)"


# 중복된 부분 찾기 (1) 책이름과 저자명이 완벽히 일치

In [9]:
# 중복된 부분 찾기 (1) 책이름과 저자명이 완벽히 일치
for student in df.name.unique():
  #학생별로 도서명 문자열로 담기
  temp = df[df.name==student]
  all_book = temp.book.tolist()
  book_list_incomplete = []
  for book_by_row in all_book :   
    book_list_incomplete = book_list_incomplete+book_by_row.split("), ")

  #print("1. " , book_list_incomplete)

  # 빈 문자열 원소 제거 및 괄호 처리하기
  book_list = []
  for book in book_list_incomplete:
    if len(book)==0:
      continue
    elif book[-1]==")":
      book_list.append(book)
    else:
      book_list.append(book+")")
  #print("2. " , book_list)

  # 중복 횟수 세기
  book_count={}
  lists = book_list
  for i in lists:
      try: book_count[i] += 1
      except: book_count[i]=1
  #print("3. " , book_count)

  # 중복 횟수가 2 이상인 아이템의 key만 담기
  book_duplicated = []
  for k, v in book_count.items():
      if v >= 2: 
          book_duplicated.append(k)
  #print("4. " , book_duplicated)

  # 출력하기
  if len(book_duplicated)>0:
    print('\n',student, "학생의 중복된 항목이 있습니다....:", book_duplicated)
    print('다음 영역 중에서 찾아보세요.')
    for i in range(len(book_duplicated)):
      display(temp[temp['book'].str.contains(book_duplicated[i][:2])]) 
  else:
    continue
    


 강철수 학생의 중복된 항목이 있습니다....: ['수학의 역사(지즈강)']
다음 영역 중에서 찾아보세요.


,name,section,year,grade,sem,book
1,강철수,수학,2018,1,2,수학의 역사(지즈강)
7,강철수,수학,2019,2,2,수학비타민 플러스(박경미)
9,강철수,공통,2019,2,2,"수학의 역사(지즈강), The brain(데이비드 이글먼), 열두 발자국(정재승)"



 고영희 학생의 중복된 항목이 있습니다....: ['나를 찾는 심리 탐구서(박진영)']
다음 영역 중에서 찾아보세요.


,name,section,year,grade,sem,book
21,고영희,국어,2019,2,1,"나를 찾는 심리 탐구서(박진영), 이 정도는 알아야 하는 최소한의 국제 이슈(연유진 외)"
34,고영희,국어,2020,3,2,"나를 찾는 심리 탐구서(박진영), 나는 말랄라(말랄라 유사이프, 퍼트리샤 매코믹)"


# 중복된 부분 찾기 (2) 오타 등이 포함되어있는 경우

In [10]:
# 중복된 부분 찾기 (2) 오타 등이 포함되어있는 경우
pd.options.display.max_colwidth = 1000

for student in df.name.unique():
  #학생별로 도서명 문자열로 담기
  temp = df[df.name==student]
  all_book = temp.book.tolist()
  ####################################
  book_list_incomplete = []
  for book_by_row in all_book :   
    book_split = book_by_row.replace("("," ").replace(")"," ").replace(","," ")
    book_list_incomplete = book_list_incomplete + book_split.split(" ")
  book_list_incomplete = list(filter(None, book_list_incomplete))
  
  # 중복 횟수 세기
  book_count={}
  lists = book_list_incomplete
  for i in lists:
      try: book_count[i] += 1
      except: book_count[i]=1

  book_duplicated = []
  for k, v in book_count.items():
      if (v >= 2) & (len(k)>1): 
          book_duplicated.append([k,v])

  if len(book_duplicated)>0:
    print(student, "학생의 중복된 항목이 있습니다....:", book_duplicated)
    print('다음 영역 중에서 찾아보세요.')
    for i in range(len(book_duplicated)):
      print('\n중복된 단어와 중복 횟수 : ',book_duplicated[i])
      display(temp[temp['book'].str.contains(book_duplicated[i][0])]) 
  else:
    continue


강철수 학생의 중복된 항목이 있습니다....: [['과학', 3], ['이야기', 2], ['수학의', 2], ['역사', 2], ['지즈강', 2], ['정재승', 2], ['나는', 2], ['말랄라', 2]]
다음 영역 중에서 찾아보세요.

중복된 단어와 중복 횟수 :  ['과학', 3]


,name,section,year,grade,sem,book
0,강철수,공통,2018,1,1,"서민의 기생충 열전(서미), 과학 일시정지(가치를 꿈꾸는 과학교사모임), 모든 순간의 물리학(카를로 로벨리), 미래학자의 인공지능 시나리오(최윤식), 내 휴대폰 속의 슈퍼스파이(타니아 로이드 치), 왜 세계의 절반은 굶주리는가(장 지글러), 할아버지가 들려주는 우주 이야기(위베르 리브스)"
4,강철수,과학,2019,2,1,"하리하라의 몸 이야기(이은희), 4차 산업혁명 세상을 바꾸는 14가지 미래 기술(한국경제TV 산업팀), 과학, 리플레이(가치를꿈꾸는과학교사모임), 과학 콘서트(정재승)"
13,강철수,공통,2020,3,2,"탐정이 된 과학자들(마릴리 피터스), 바이러스 쇼크(최강석)"



중복된 단어와 중복 횟수 :  ['이야기', 2]


,name,section,year,grade,sem,book
0,강철수,공통,2018,1,1,"서민의 기생충 열전(서미), 과학 일시정지(가치를 꿈꾸는 과학교사모임), 모든 순간의 물리학(카를로 로벨리), 미래학자의 인공지능 시나리오(최윤식), 내 휴대폰 속의 슈퍼스파이(타니아 로이드 치), 왜 세계의 절반은 굶주리는가(장 지글러), 할아버지가 들려주는 우주 이야기(위베르 리브스)"
4,강철수,과학,2019,2,1,"하리하라의 몸 이야기(이은희), 4차 산업혁명 세상을 바꾸는 14가지 미래 기술(한국경제TV 산업팀), 과학, 리플레이(가치를꿈꾸는과학교사모임), 과학 콘서트(정재승)"



중복된 단어와 중복 횟수 :  ['수학의', 2]


...

,name,section,year,grade,sem,book
110,마성수,국어,2019,2,2,"달처럼 동그란 내 얼굴(미레유 디스데로), 나는 초콜릿의 달콤함을 모릅니다(타라 설리번)"
120,마성수,국어,2020,3,2,"나는 말랄라(말랄라 유사이프, 퍼트리샤 매코믹)"



중복된 단어와 중복 횟수 :  ['해설과', 2]


,name,section,year,grade,sem,book
113,마성수,공통,2019,2,2,"해설과 함께 읽는 빈처/벙어리 삼룡이 외(전도연 엮음), 이름을 빼앗긴 소녀 에바(조안 M. 울프), 구운몽(김만중)"
119,마성수,공통,2020,3,1,"권오숙 교수의 해설과 함께 읽는 베니스의 상인(윌리엄 셰익스피어), 설중환 교수와 함께 읽는 춘향전(설중환), 엄광용 선생님과 함께 읽는 삼국유사(일연)"



중복된 단어와 중복 횟수 :  ['말랄라', 2]


,name,section,year,grade,sem,book
120,마성수,국어,2020,3,2,"나는 말랄라(말랄라 유사이프, 퍼트리샤 매코믹)"


# 중복된 부분 찾기 (3) 자연어 처리(kiwi)

In [24]:
#to be continued